In [1]:
import pandas as pd
import json
import requests
from warcio import ArchiveIterator
import os 
import numpy as np 
from multiprocessing import set_start_method
from multiprocessing import get_context
from multiprocessing import pool 
import multiprocessing
from requests.adapters import HTTPAdapter
from requests.packages import urllib3
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm 
import pickle 
import re

In [2]:
trial2 = pd.read_pickle('trial_28_02.pkl')

In [16]:
sys.getsizeof(trial2)

981194

# START HAVING A LOOK AT GETTING POSTCODES

In [88]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    return list(postcodes) 

In [101]:
text = 'E2 6DZ CB1 7ED'

In [89]:
def lambda_postcode(df):
    df['postcode'] = df.apply(lambda x: postcode_finder(x['text']), axis = 1)
    return df 

In [98]:
def lambda_postcode(df):
    df['postcode'] = df['text'].apply(lambda x: postcode_finder(x))
    return df 

In [5]:
trial3= trial2.dropna(subset='text')

In [11]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    return list(postcodes) 
def postcode_counter_webpage(text):
    postcodes = postcode_finder(text)
    webpage_postcodes = [postcode[1] for postcode in postcodes]
    dictionary_with_counts = {x:webpage_postcodes.count(x) for x in webpage_postcodes}
    return dictionary_with_counts
def lambda_counter_webpage(df):
    df['postcode'] = df['text'].apply(lambda x: postcode_counter_webpage(x))
    return df
#sta prendendo text da giiu sotto, lol. 

In [12]:
trial4 = trial3[80:91]

In [13]:
trial4

,url,url_host_name,url_host_registry_suffix,url_host_registered_domain,url_host_private_suffix,url_host_private_domain,fetch_time,fetch_status,fetch_redirect,content_mime_type,content_charset,content_languages,content_truncated,warc_filename,warc_record_offset,warc_record_length,warc_segment,needed_warc,text
81,https://www.001-translations.co.uk/c_10/servic...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 01:13:40,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,538045750,5104,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Website translation - 001 Translation UK\n00...
82,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:46:10,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,571777495,4697,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
83,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:02:57,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,584262048,4862,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
84,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-13 23:46:46,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,563143726,4859,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
85,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:25:27,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,560176613,4859,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
86,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:21:06,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,581835915,4877,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
87,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:58:23,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,553357820,4862,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
88,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 01:19:06,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,580981523,4873,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
89,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:11:15,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,573095298,4866,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...
90,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:14:35,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,559901207,4877,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,

In [14]:
df = lambda_counter_webpage(trial4)

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_33038/3855214437.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['postcode'] = df['text'].apply(lambda x: postcode_counter_webpage(x))


In [15]:
df

,url,url_host_name,url_host_registry_suffix,url_host_registered_domain,url_host_private_suffix,url_host_private_domain,fetch_time,fetch_status,fetch_redirect,content_mime_type,content_charset,content_languages,content_truncated,warc_filename,warc_record_offset,warc_record_length,warc_segment,needed_warc,text,postcode
81,https://www.001-translations.co.uk/c_10/servic...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 01:13:40,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,538045750,5104,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Website translation - 001 Translation UK\n00...,{}
82,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:46:10,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,571777495,4697,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{}
83,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:02:57,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,584262048,4862,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'BT9 6DP': 1}
84,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-13 23:46:46,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,563143726,4859,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'LU1 3FL': 1}
85,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:25:27,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,560176613,4859,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'CB1 7ED': 1}
86,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:21:06,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,581835915,4877,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'WD17 4PT': 1}
87,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:58:23,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,553357820,4862,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'M1 1EL': 1}
88,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 01:19:06,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,580981523,4873,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'NR6 6HP': 1}
89,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:11:15,200,None,text/html,UTF-8,eng,None,crawl-data/CC-MAIN-2022-33/segments/1659882571...,573095298,4866,1659882571989.67,https://data.commoncrawl.org/crawl-data/CC-MAI...,﻿ Contact - 001 Translation UK\n001 Translatio...,{'NG3 5JU': 1}
90,https://www.001-translations.co.uk/c_132/conta...,www.001-translations.co.uk,co.uk,001-translations.co.uk,co.uk,001-translations.co.uk,2022-08-14 00:14:35,200,None,text/html,UTF-8,eng,None,crawl-data/C